### Part 2: Programming Problems

#### Problem 1 – Line Search
##### (1) Define the function $f(x)$

We consider the function
$$
f(x)= x_1^2+2x_2^2,
$$
where $x=(x_1,x_2)\in\mathbb{R}^2$.

In [7]:
import numpy as np

def f_line(x):
    # x is a numpy array: [x1, x2]
    x1, x2 = x[0], x[1]
    return x1**2 + 2*x2**2

##### (2) Define the gradient of $f(x)$

The gradient is
$$
\nabla f(x)= \begin{pmatrix} 2x_1 \\ 4x_2 \end{pmatrix}.
$$

In [8]:
def grad_f_line(x):
    x1, x2 = x[0], x[1]
    return np.array([2*x1, 4*x2])

##### (3) Evaluate $f(x)$ and $\nabla f(x)$ at $x^{(0)}=(9,1)^T$

In [9]:
x0_line = np.array([9, 1])
print("f(x0) =", f_line(x0_line))
print("grad f(x0) =", grad_f_line(x0_line))

f(x0) = 83
grad f(x0) = [18  4]


##### (4) Define the function $wolfe\_conditions$

This function verifies if the Wolfe conditions hold for a given step size $\alpha>0$. It takes:
- A function $f$
- Its gradient function $\nabla f$
- A current point $x$
- A descent direction $d$
- A step size $\alpha$
- Parameters $\eta$ and $\bar{\eta}$

and returns a tuple of booleans $(\text{first\_wolfe}, \text{second\_wolfe})$.

The Wolfe conditions are:
1. First Wolfe condition (sufficient decrease):
$$
f(x+\alpha d)\le f(x)+\alpha\eta\,\nabla f(x)^T d.
$$
2. Second Wolfe condition (curvature):
$$
\nabla f(x+\alpha d)^T d \ge \bar{\eta}\,\nabla f(x)^T d.
$$

In [10]:
def wolfe_conditions(f, grad_f, x, d, alpha, eta, eta_bar):
    cond1 = f(x + alpha * d) <= f(x) + alpha * eta * np.dot(grad_f(x), d)
    cond2 = np.dot(grad_f(x + alpha * d), d) >= eta_bar * np.dot(grad_f(x), d)
    return (cond1, cond2)


##### (5) Test the $wolfe\_conditions$ function

Test for $f(x)=x_1^2+2x_2^2$ at $x^{(0)}=(9,1)^T$ with 
$$
d^{(0)}=-\nabla f(x^{(0)}),
$$ 
$\alpha=0.05$, $\eta=0.01$, and $\bar{\eta}=0.8$.

In [11]:
d0_line = -grad_f_line(x0_line)
wolfe_test = wolfe_conditions(f_line, grad_f_line, x0_line, d0_line, 0.05, 0.01, 0.8)
print("Wolfe conditions (first, second):", wolfe_test)

Wolfe conditions (first, second): (True, False)


##### (6) Define the function $backtracking$

Implement the backtracking line search. It takes:
- A function $f$
- Its gradient $\nabla f$
- A current point $x$
- A descent direction $d$
- A maximum step size $\bar{\alpha}$
- Parameter $\eta$ (for the first Wolfe condition)

and returns the step size $\alpha^*$ and the new point $x_{\text{new}} = x+\alpha^* d$.  
The function should use the $wolfe\_conditions$ function.

In [14]:
def backtracking(f, grad_f, x, d, alpha_bar, eta, eta_bar=0.8, tau=0.5):
    alpha = alpha_bar
    while True:
        cond1, cond2 = wolfe_conditions(f, grad_f, x, d, alpha, eta, eta_bar)
        if cond1 and cond2:
            break
        alpha *= tau
        if alpha < 1e-8:
            break
    x_new = x + alpha * d
    return alpha, x_new

##### (7) Test the Backtracking Function

Test the backtracking function for $f(x)=x_1^2+2x_2^2$ at $x^{(0)}=(9,1)^T$ with 
$$
d^{(0)}=-\nabla f(x^{(0)}),
$$ 
$\bar{\alpha}=10$, and $\eta=0.01$.

In [16]:
alpha_star, x_new_line = backtracking(f_line, grad_f_line, x0_line, d0_line, alpha_bar=10, eta=0.01)
print("Test Backtracking: α* =", alpha_star)
print("Test Backtracking: x_new =", x_new_line)

Test Backtracking: α* = 0.625
Test Backtracking: x_new = [-2.25 -1.5 ]


#### Problem 2 – Unconstrained Optimisation
##### (1) Define the function

We consider:
$$
f(x)= x_1^3 - x_1 + x_2^3 - x_2.
$$

In [ ]:
def f_uncon(x):
    # x is a numpy array [x1, x2]
    x1, x2 = x[0], x[1]
    return x1**3 - x1 + x2**3 - x2

##### (2) Define the gradient of $f(x)$

For $x_1^3-x_1$, the derivative is $3x_1^2-1$.  
For $x_2^3-x_2$, the derivative is $3x_2^2-1$.  

Thus,
$$
\nabla f(x)= \begin{pmatrix} 3x_1^2-1 \\ 3x_2^2-1 \end{pmatrix}.
$$

In [ ]:
def grad_f_uncon(x):
    # Compute gradient components for f(x)=x1^3-x1+x2^3-x2
    x1, x2 = x[0], x[1]
    return np.array([3*x1**2 - 1, 3*x2**2 - 1])

##### (3) Fixed-step Gradient Descent

We implement gradient descent with the update rule:
$$
x^{(k+1)} = x^{(k)} - \alpha\,\nabla f(x^{(k)}),
$$
with a constant step size $\alpha$.  
We use the starting point $x^{(0)}=(1,1)^T$.

In [19]:
def fixed_gd(f, grad_f, x_start, alpha, max_iter=1000, tol=1e-6):
    # Initialize current point and store iterates
    x_current = np.array(x_start, dtype=float)
    iterates = [x_current.copy()]
    for _ in range(max_iter):
        grad = grad_f(x_current)
        # Stop if the gradient norm is below tolerance
        if np.linalg.norm(grad) < tol:
            break
        # Update rule for gradient descent
        x_current = x_current - alpha * grad
        iterates.append(x_current.copy())
    return x_current, iterates

x0_uncon = np.array([1, 1])
alpha_fixed = 0.001  # Chosen fixed step size
x_fixed, iterates_fixed = fixed_gd(f_uncon, grad_f_uncon, x0_uncon, alpha_fixed)
print("Fixed-step GD final x:", x_fixed)
print("Fixed-step GD f(x):", f_uncon(x_fixed))

Fixed-step GD final x: [0.5870474 0.5870474]
Fixed-step GD f(x): -0.7694727904830598
